# Notebook 06: Semantic QA Evaluation

## Objective

Compare search methods vs graph traversal on semantic QA.

## Evaluation Methods

1. **Search-based** (like v2): Use hybrid search to find answers
2. **Graph traversal**: Use one-hop to traverse from source entity

## Expected Findings

- Search: Lower EM on semantic QA than v2's 95.2% on vocabulary QA
- Graph: Higher recall (this is what graphs are designed for)

In [1]:
# Standard imports
import sys
import time
from pathlib import Path
from datetime import datetime
from collections import Counter, defaultdict

# Add project root to path
PROJECT_ROOT = Path.cwd().parents[1]
sys.path.insert(0, str(PROJECT_ROOT / 'src'))
sys.path.insert(0, str(Path.cwd()))

# Import utilities
from kg_o1_v3_utils import (
    save_json, load_json,
    hybrid_search, text_search, test_one_hop, parse_one_hop_edges,
    find_answer_rank, compute_recall_at_k, compute_mrr, compute_exact_match,
    assess_statistical_validity,
)

# Output directory
OUTPUT_DIR = Path.cwd() / 'outputs'
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Output directory: {OUTPUT_DIR}")

Project root: /home/trentleslie/Insync/projects/biomapper2
Output directory: /home/trentleslie/Insync/projects/biomapper2/notebooks/kg_o1_v3/outputs


## 1. Load Semantic QA Dataset

In [2]:
# Load QA dataset from NB05
qa_file = OUTPUT_DIR / 'semantic_qa_dataset.json'

if qa_file.exists():
    qa_data = load_json(qa_file)
    qa_pairs = qa_data.get('qa_pairs', [])
    print(f"Loaded {len(qa_pairs)} QA pairs from NB05")
    
    # Show summary
    summary = qa_data.get('summary', {})
    print(f"\nDataset summary:")
    print(f"  - 1-hop QA: {summary.get('one_hop_qa', 0)}")
    print(f"  - Multi-hop QA: {summary.get('multi_hop_qa', 0)}")
    print(f"  - Validation rate: {100*summary.get('validation_rate', 0):.1f}%")
else:
    print("WARNING: NB05 output not found. Run NB05 first.")
    qa_pairs = []

Loaded 3359 QA pairs from NB05

Dataset summary:
  - 1-hop QA: 3356
  - Multi-hop QA: 3
  - Validation rate: 0.0%


## 2. Evaluate Search-Based Method

Use hybrid search (like v2) to retrieve answers.

In [3]:
# Evaluate search-based retrieval
print("="*60)
print("Evaluating SEARCH-BASED method")
print("="*60)

search_results = []

for i, qa in enumerate(qa_pairs):
    if i % 20 == 0:
        print(f"  Evaluating {i+1}/{len(qa_pairs)}...")
    
    question = qa.get('question', '')
    answer_id = qa.get('answer_id', '')
    answer_name = qa.get('answer', '')
    
    # Search using the question
    start_time = time.time()
    results = hybrid_search(question, limit=20)
    latency_ms = (time.time() - start_time) * 1000
    
    # Find rank of answer
    rank = find_answer_rank(results, answer_id)
    
    # Also try searching for answer name directly
    if rank == 0:
        rank = find_answer_rank(results, answer_name)
    
    search_results.append({
        'question': question,
        'answer_id': answer_id,
        'answer_name': answer_name,
        'rank': rank,
        'found': rank > 0,
        'exact_match': rank == 1,
        'latency_ms': latency_ms,
        'qa_type': qa.get('qa_type', ''),
        'num_hops': qa.get('num_hops', 1),
    })
    
    # Rate limiting
    time.sleep(0.1)

print(f"\nEvaluated {len(search_results)} QA pairs with search")

Evaluating SEARCH-BASED method
  Evaluating 1/3359...


  Evaluating 21/3359...


  Evaluating 41/3359...


  Evaluating 61/3359...


  Evaluating 81/3359...


...

  Evaluating 3281/3359...


  Evaluating 3301/3359...


  Evaluating 3321/3359...


  Evaluating 3341/3359...



Evaluated 3359 QA pairs with search


In [4]:
# Calculate search metrics
search_ranks = [r['rank'] for r in search_results]

search_metrics = {
    'method': 'hybrid_search',
    'n': len(search_results),
    'exact_match': compute_exact_match(search_ranks),
    'recall_at_5': compute_recall_at_k(search_ranks, 5),
    'recall_at_10': compute_recall_at_k(search_ranks, 10),
    'recall_at_20': compute_recall_at_k(search_ranks, 20),
    'mrr': compute_mrr(search_ranks),
    'avg_latency_ms': sum(r['latency_ms'] for r in search_results) / len(search_results) if search_results else 0,
}

print("\nSearch Method Metrics:")
print(f"  Exact Match (EM): {100*search_metrics['exact_match']:.1f}%")
print(f"  Recall@5: {100*search_metrics['recall_at_5']:.1f}%")
print(f"  Recall@10: {100*search_metrics['recall_at_10']:.1f}%")
print(f"  Recall@20: {100*search_metrics['recall_at_20']:.1f}%")
print(f"  MRR: {search_metrics['mrr']:.3f}")
print(f"  Avg latency: {search_metrics['avg_latency_ms']:.1f}ms")


Search Method Metrics:
  Exact Match (EM): 0.0%
  Recall@5: 0.1%
  Recall@10: 0.1%
  Recall@20: 0.2%
  MRR: 0.001
  Avg latency: 2.8ms


## 3. Evaluate Graph Traversal Method

Use one-hop from source entity to find answers.

In [5]:
# Evaluate graph-based retrieval
print("="*60)
print("Evaluating GRAPH TRAVERSAL method")
print("="*60)

graph_results = []

for i, qa in enumerate(qa_pairs):
    if i % 20 == 0:
        print(f"  Evaluating {i+1}/{len(qa_pairs)}...")
    
    source_entity_id = qa.get('source_entity_id', '')
    answer_id = qa.get('answer_id', '') or ''
    answer_name = qa.get('answer', '') or ''
    
    # Get predicate from reasoning chain
    chain = qa.get('reasoning_chain', [])
    predicate = chain[0].get('predicate', '') if chain else None
    
    # Graph traversal from source entity
    start_time = time.time()
    
    if source_entity_id:
        # Try with predicate filter first
        result = test_one_hop(source_entity_id, predicate_filter=predicate) if predicate else test_one_hop(source_entity_id)
        
        # Use parse_one_hop_edges to correctly extract edges with predicates
        edges = parse_one_hop_edges(result)
        
        # Check if answer is in edges
        found = False
        for edge in edges:
            obj_id = (edge.get('object_id') or edge.get('end_node_id') or '').lower()
            obj_name = (edge.get('object_name') or edge.get('end_node_name') or '').lower()
            
            if answer_id.lower() == obj_id or answer_name.lower() in obj_name or answer_name.lower() == obj_name:
                found = True
                break
    else:
        found = False
        edges = []
    
    latency_ms = (time.time() - start_time) * 1000
    
    graph_results.append({
        'source_entity_id': source_entity_id,
        'answer_id': answer_id,
        'answer_name': answer_name,
        'found': found,
        'num_edges': len(edges),
        'latency_ms': latency_ms,
        'qa_type': qa.get('qa_type', ''),
        'num_hops': qa.get('num_hops', 1),
    })
    
    time.sleep(0.1)

print(f"\nEvaluated {len(graph_results)} QA pairs with graph traversal")

Evaluating GRAPH TRAVERSAL method
  Evaluating 1/3359...


ERROR:root:Kestrel API HTTP error (one-hop): 400 Client Error: Bad Request for url: https://kestrel.nathanpricelab.com/api/one-hop
Traceback (most recent call last):
  File "/home/trentleslie/Insync/projects/biomapper2/src/biomapper2/utils.py", line 130, in kestrel_request
    response.raise_for_status()
  File "/home/trentleslie/Insync/projects/biomapper2/.venv/lib/python3.12/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://kestrel.nathanpricelab.com/api/one-hop


ERROR:root:Kestrel API HTTP error (one-hop): 400 Client Error: Bad Request for url: https://kestrel.nathanpricelab.com/api/one-hop
Traceback (most recent call last):
  File "/home/trentleslie/Insync/projects/biomapper2/src/biomapper2/utils.py", line 130, in kestrel_request
    response.raise_for_status()
  File "/home/trentleslie/Insync/projects/biomapper2/.venv/lib/python3.12/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://kestrel.nathanpricelab.com/api/one-hop


  Evaluating 21/3359...


  Evaluating 41/3359...


...

  Evaluating 3281/3359...


  Evaluating 3301/3359...


  Evaluating 3321/3359...


  Evaluating 3341/3359...



Evaluated 3359 QA pairs with graph traversal


In [6]:
# Calculate graph metrics
graph_recall = sum(1 for r in graph_results if r['found']) / len(graph_results) if graph_results else 0

graph_metrics = {
    'method': 'graph_traversal',
    'n': len(graph_results),
    'recall': graph_recall,
    'avg_edges_examined': sum(r['num_edges'] for r in graph_results) / len(graph_results) if graph_results else 0,
    'avg_latency_ms': sum(r['latency_ms'] for r in graph_results) / len(graph_results) if graph_results else 0,
}

print("\nGraph Traversal Metrics:")
print(f"  Recall: {100*graph_metrics['recall']:.1f}%")
print(f"  Avg edges examined: {graph_metrics['avg_edges_examined']:.1f}")
print(f"  Avg latency: {graph_metrics['avg_latency_ms']:.1f}ms")


Graph Traversal Metrics:
  Recall: 63.2%
  Avg edges examined: 54.9
  Avg latency: 27.7ms


## 4. Compare Methods

In [7]:
# Compare methods
print("="*60)
print("METHOD COMPARISON")
print("="*60)

print(f"\n{'Metric':<25} {'Search':<15} {'Graph':<15}")
print("-" * 55)
print(f"{'Exact Match / Recall':<25} {100*search_metrics['exact_match']:.1f}%{'':<10} {100*graph_metrics['recall']:.1f}%")
print(f"{'Recall@5 / -':<25} {100*search_metrics['recall_at_5']:.1f}%{'':<10} -")
print(f"{'Recall@10 / -':<25} {100*search_metrics['recall_at_10']:.1f}%{'':<10} -")
print(f"{'MRR / -':<25} {search_metrics['mrr']:.3f}{'':<10} -")
print(f"{'Avg Latency (ms)':<25} {search_metrics['avg_latency_ms']:.1f}{'':<10} {graph_metrics['avg_latency_ms']:.1f}")

METHOD COMPARISON

Metric                    Search          Graph          
-------------------------------------------------------
Exact Match / Recall      0.0%           63.2%
Recall@5 / -              0.1%           -
Recall@10 / -             0.1%           -
MRR / -                   0.001           -
Avg Latency (ms)          2.8           27.7


## 5. Performance by QA Type

In [8]:
# Performance by QA type
print("="*60)
print("PERFORMANCE BY QA TYPE")
print("="*60)

# Group by QA type
search_by_type = defaultdict(list)
graph_by_type = defaultdict(list)

for r in search_results:
    search_by_type[r['qa_type']].append(r)

for r in graph_results:
    graph_by_type[r['qa_type']].append(r)

print(f"\n{'QA Type':<20} {'N':<6} {'Search EM':<12} {'Graph Recall':<12}")
print("-" * 50)

for qa_type in sorted(search_by_type.keys()):
    search_type_results = search_by_type[qa_type]
    graph_type_results = graph_by_type[qa_type]
    
    n = len(search_type_results)
    search_em = sum(1 for r in search_type_results if r['exact_match']) / n if n > 0 else 0
    graph_recall = sum(1 for r in graph_type_results if r['found']) / len(graph_type_results) if graph_type_results else 0
    
    print(f"{qa_type:<20} {n:<6} {100*search_em:.1f}%{'':<8} {100*graph_recall:.1f}%")

PERFORMANCE BY QA TYPE

QA Type              N      Search EM    Graph Recall
--------------------------------------------------
semantic_1_hop       3356   0.0%         63.3%
semantic_3_hop       3      0.0%         0.0%


## 6. Performance by Number of Hops

In [9]:
# Performance by number of hops
print("="*60)
print("PERFORMANCE BY NUMBER OF HOPS")
print("="*60)

search_by_hops = defaultdict(list)
graph_by_hops = defaultdict(list)

for r in search_results:
    search_by_hops[r['num_hops']].append(r)

for r in graph_results:
    graph_by_hops[r['num_hops']].append(r)

print(f"\n{'Hops':<10} {'N':<6} {'Search EM':<12} {'Graph Recall':<12}")
print("-" * 40)

for num_hops in sorted(search_by_hops.keys()):
    search_hop_results = search_by_hops[num_hops]
    graph_hop_results = graph_by_hops[num_hops]
    
    n = len(search_hop_results)
    search_em = sum(1 for r in search_hop_results if r['exact_match']) / n if n > 0 else 0
    graph_recall = sum(1 for r in graph_hop_results if r['found']) / len(graph_hop_results) if graph_hop_results else 0
    
    print(f"{num_hops:<10} {n:<6} {100*search_em:.1f}%{'':<8} {100*graph_recall:.1f}%")

PERFORMANCE BY NUMBER OF HOPS

Hops       N      Search EM    Graph Recall
----------------------------------------
1          3356   0.0%         63.3%
3          3      0.0%         0.0%


## 7. Statistical Validity Assessment

In [10]:
# Statistical validity
print("="*60)
print("STATISTICAL VALIDITY ASSESSMENT")
print("="*60)

validity = assess_statistical_validity(len(qa_pairs))

print(f"\nSample size: {validity['n']}")
print(f"Can compute CI: {validity['can_compute_ci']}")
print(f"Per-group valid: {validity['per_group_valid']}")
print(f"Fully reliable: {validity['fully_reliable']}")
print(f"\nInterpretation: {validity['interpretation']}")

STATISTICAL VALIDITY ASSESSMENT

Sample size: 3359
Can compute CI: True
Per-group valid: True
Fully reliable: True

Interpretation: Statistically reliable claims supported


## 8. Save Evaluation Results

In [11]:
# Save evaluation results
output_data = {
    'timestamp': datetime.now().isoformat(),
    'summary': {
        'n_qa_pairs': len(qa_pairs),
        'statistical_validity': validity,
    },
    'search_method': {
        'metrics': search_metrics,
        'results': search_results,
    },
    'graph_method': {
        'metrics': graph_metrics,
        'results': graph_results,
    },
    'comparison': {
        'search_em': search_metrics['exact_match'],
        'graph_recall': graph_metrics['recall'],
        'insight': (
            'Graph traversal outperforms search' if graph_metrics['recall'] > search_metrics['exact_match']
            else 'Search outperforms graph traversal' if search_metrics['exact_match'] > graph_metrics['recall']
            else 'Methods perform similarly'
        ),
    },
}

save_json(output_data, OUTPUT_DIR / 'semantic_qa_evaluation.json')
print(f"\nEvaluation results saved to: {OUTPUT_DIR / 'semantic_qa_evaluation.json'}")


Evaluation results saved to: /home/trentleslie/Insync/projects/biomapper2/notebooks/kg_o1_v3/outputs/semantic_qa_evaluation.json


## Summary

In [12]:
# Final summary
print("\n" + "="*60)
print("NOTEBOOK 06 COMPLETE")
print("="*60)

print(f"\nEvaluation Results:")
print(f"  - QA pairs evaluated: {len(qa_pairs)}")
print(f"  - Search Exact Match: {100*search_metrics['exact_match']:.1f}%")
print(f"  - Graph Recall: {100*graph_metrics['recall']:.1f}%")
print(f"\nInsight: {output_data['comparison']['insight']}")

print(f"\nv2 comparison:")
print(f"  - v2 achieved 95.2% EM on vocabulary QA")
print(f"  - v3 search achieves {100*search_metrics['exact_match']:.1f}% EM on semantic QA")

if search_metrics['exact_match'] < 0.5:
    print(f"\nThis confirms: Search is weaker on semantic QA than vocabulary QA.")
    print(f"Graph traversal may be better for multi-hop reasoning.")

print(f"\nNext step: NB07 - Semantic vs Vocabulary Gap Analysis")


NOTEBOOK 06 COMPLETE

Evaluation Results:
  - QA pairs evaluated: 3359
  - Search Exact Match: 0.0%
  - Graph Recall: 63.2%

Insight: Graph traversal outperforms search

v2 comparison:
  - v2 achieved 95.2% EM on vocabulary QA
  - v3 search achieves 0.0% EM on semantic QA

This confirms: Search is weaker on semantic QA than vocabulary QA.
Graph traversal may be better for multi-hop reasoning.

Next step: NB07 - Semantic vs Vocabulary Gap Analysis
